In [ ]:
import open3d as o3d
import tensorflow as tf
import numpy as np
import os
from utilities import *
import re
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import glob
from pyntcloud import PyntCloud
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from plyfile import PlyData, PlyElement
from scipy.spatial import cKDTree
from tqdm.notebook import tqdm, trange
from multiprocessing import Pool

In [ ]:
initial_path = r'write the path where you put the project'
noisy_modelnet_pcs_path = os.path.join(initial_path, r'noisy_modelnet_pcs')
db_path = os.path.join(initial_path, r'icip2020_perry_quality_repack_bin')
deg_metrics_path = os.path.join('data', 'icip2020_deg_metrics.json')
degraded_pcs_features_path = os.path.join('data', 'icip2020_degraded_pcs_features.csv')
degraded_pcs_features_preds_path = os.path.join('data', 'icip2020_degraded_pcs_features_preds.csv')
block_bits = 6
block_shape = [2**block_bits] * 3
bbox_min = [0,0,0]

# Training on ModelNet dataset

In [ ]:
train_glob = os.path.join(initial_path, r'ModelNet40_200_pc512_oct3_4k/**/*.ply')
files = get_files(train_glob)
assert len(files) > 0
original_modelnet_pcs = []
for path in tqdm(files):
    pc = modelnet_pc(path)
    pc.load_points(path)
    pc.load_tree()
    pc.compute_normals()
    original_modelnet_pcs.append(pc)

In [ ]:
std_list=[]
for k in range(1):
    for i in range(5):
        for j in range(5):
            std_list.append(i+((0.2)*(j+1)))

In [ ]:
noisy_modelnet_pcs = []
d2_list=[]
for pc in tqdm(original_modelnet_pcs) :
    for std in std_list:
        new_pc = make_noisy_version(pc , std)
        new_pc.load_tree()
        new_pc.load_dists_ngbs()
        new_pc.compute_normals()
        new_pc.compute_features()
        d2 = new_pc.features['min_d2_psnr']
        d2_list.append(d2)
        noisy_modelnet_pcs.append(pc.content + '__' + pc.pc_name+ '__' + str(std) + '__' + str(d2))
        new_pc.write_to_disk(noisy_modelnet_pcs_path, d2)

In [ ]:
textfile = open(os.path.join(initial_path,"info_list_noisy_modelnet_pcs.txt"), "w")
for element in noisy_modelnet_pcs :
    textfile.write(element + "\n")
textfile.close()

In [ ]:
noisy_modelnet_pcs = []
L = open(os.path.join(initial_path,"info_list_noisy_modelnet_pcs.txt"), "r").read().splitlines();
for line in L:
  noisy_modelnet_pcs.append(line)

In [ ]:
def shuffle_and_split(number, train_ratio):
    randomlist = np.arange(number)
    np.random.shuffle(randomlist)
    train_randomlist = randomlist[0:round(number*(1-train_ratio))]
    set_randomlist = set(randomlist)
    set_train_randomlist = set(train_randomlist) 
    validation_randomlist = set_randomlist-set_train_randomlist
    train_names = np.array(list(set_train_randomlist))
    np.random.shuffle(train_names)
    validation_names = np.array(list(validation_randomlist))
    np.random.shuffle(validation_names)
    return train_names, validation_names


In [ ]:
def push_example (num): 
    names_list = noisy_modelnet_pcs
    dataset_path = noisy_modelnet_pcs_path
    x1_path = names_list[num] 
    x1 = PyntCloud.from_file(os.path.join(dataset_path, x1_path))
    x1_points = x1.points[['x','y','z']].to_numpy()
    x1 = o3d.geometry.PointCloud()
    x1.points = o3d.utility.Vector3dVector(x1_points)
    R1 = x1.get_rotation_matrix_from_xyz((0, 0, np.random.uniform(0, 2) * np.pi))
    x1.rotate(R1, center = (32,32,32))
    x1.scale(np.random.uniform(0.8,1), center = x1.get_center())
    x1 = np.clip(np.asarray(x1.points), 0, 63)#.to_numpy()
    zeros1 = np.zeros(block_shape, dtype=np.float32)
    x1 = pts_to_vx(x1, block_shape, zeros1)
    x1 = x1.reshape([64,64,64,1])
    for pc in original_modelnet_pcs: 
        if names_list[num].split('__')[2] == pc.pc_name:
            x2_points = pc.points[['x','y','z']].to_numpy()
            x2 = o3d.geometry.PointCloud()
            x2.points = o3d.utility.Vector3dVector(x2_points)
            R2 = x2.get_rotation_matrix_from_xyz((0, 0, np.random.uniform(0, 2) * np.pi))
            x2.rotate(R2, center = (32,32,32))
            x2.scale(np.random.uniform(0.8, 1), center = x2.get_center())
            x2 = np.clip(np.asarray(x2.points), 0, 63)
            break
    zeros2 = np.zeros(block_shape, dtype=np.float32)
    x2 = pts_to_vx(x2, block_shape, zeros2)
    x2 = x2.reshape([64,64,64,1])
    d2 = float(names_list[num].split('__')[-1][:-4])/75
    return x1, x2, d2

def tf_push_example (num):
    return tf.py_function(push_example,[num], [tf.float32, tf.float32, tf.float64])
def divide_example (x1, x2, d2):
    return ((x1, x2), d2)

In [ ]:
train_index, val_index = shuffle_and_split(len(noisy_modelnet_pcs), 0.1)
train_modelnet_dataset = tf.data.Dataset.from_tensor_slices(train_index)#.take(2)
train_modelnet_dataset = train_modelnet_dataset.map(tf_push_example, num_parallel_calls = 64)
train_modelnet_dataset = train_modelnet_dataset.map(divide_example, num_parallel_calls = 64)
train_modelnet_dataset = train_modelnet_dataset.batch(64)
train_modelnet_dataset = train_modelnet_dataset.prefetch(1)
val_modelnet_dataset = tf.data.Dataset.from_tensor_slices(val_index)
val_modelnet_dataset = val_modelnet_dataset.map(tf_push_example, num_parallel_calls = 64)#.take(2)
val_modelnet_dataset = val_modelnet_dataset.map(divide_example, num_parallel_calls = 64)
val_modelnet_dataset = val_modelnet_dataset.batch(64)
val_modelnet_dataset = val_modelnet_dataset.prefetch(1)

In [ ]:
filters=32
block_shape_modified=(64,64,64,1)
params = {'strides': (2, 2, 2), 'padding': 'same', 'use_bias': True}
Embedding = tf.keras.Sequential()
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_0', filters=32, kernel_size=(5, 5, 5), **params, input_shape=block_shape_modified))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_1', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_2', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_3', filters=8, kernel_size=(1, 1, 1), activation= tf.keras.activations.relu,strides=(1,1,1)))
Embedding.add(tf.keras.layers.Flatten(name='flatten'))

In [ ]:
right_input = tf.keras.Input((64,64,64,1))
left_input = tf.keras.Input((64,64,64,1))
right_y = Embedding(right_input)
left_y = Embedding(left_input)
b = tf.keras.layers.Concatenate()([right_y, left_y]) 
b=tf.keras.layers.Dense(32,activation='relu')(b)
b=tf.keras.layers.Dense(4,activation='relu')(b)
b=tf.keras.layers.Dense(1,activation='relu')(b)
Siamese = tf.keras.Model(inputs = [right_input, left_input ], outputs = [b], name="siamese")

In [ ]:
checkpoint_filepath = os.path.join(initial_path, r'chekpoints/experience_2_siamese')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
    )
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3)
Siamese.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-02), loss=tf.keras.losses.MeanSquaredError()) 

In [ ]:
history=Siamese.fit(train_modelnet_dataset, epochs=100, callbacks=[callback,reduce_lr,checkpoint_callback], initial_epoch=0 ,validation_data=val_modelnet_dataset)

## Training the Pointnet network (taken from Keras website)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import glob
import trimesh
NUM_POINTS = 2048
NUM_CLASSES = 10
BATCH_SIZE = 32

In [ ]:
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

def parse_dataset(num_points=2048):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    class_map = {}
    folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points))
            train_labels.append(i)

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )
    
train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(
    NUM_POINTS
)

def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)


In [ ]:
"""
### Build a model
Each convolution and fully-connected layer (with exception for end layers) consits of
Convolution / Dense -> Batch Normalization -> ReLU Activation.
"""

def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


"""
PointNet consists of two core components. The primary MLP network, and the transformer
net (T-net). The T-net aims to learn an affine transformation matrix by its own mini
network. The T-net is used twice. The first time to transform the input features (n, 3)
into a canonical representation. The second is an affine transformation for alignment in
feature space (n, 3). As per the original paper we constrain the transformation to be
close to an orthogonal matrix (i.e. ||X*X^T - I|| = 0).
"""


class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))


"""
 We can then define a general function to build T-net layers.
"""


def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])


"""
The main network can be then implemented in the same manner where the t-net mini models
can be dropped in a layers in the graph. Here we replicate the network architecture
published in the original paper but with half the number of weights at each layer as we
are using the smaller 10 class ModelNet dataset.
"""

inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

pointnet = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")


In [ ]:
pointnet.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

pointnet.fit(train_dataset, epochs=20, validation_data=test_dataset)

## Fine tuning and testing on ICIP 2020 dataset

In [ ]:
def build_model(num_inputs, num_dense):
    layers = []
    inputs = []
    p_input=tf.keras.Input(shape=(2048, 3),name='point_input')
    for i in range(num_inputs):
        right_input = tf.keras.Input((64,64,64,1))
        left_input = tf.keras.Input((64,64,64,1))
        siamese =  Siamese([right_input, left_input])
        layers.append(siamese)
        inputs.append(right_input)
        inputs.append(left_input)
    inputs.append(p_input)
    layers.append(pointnet(p_input))
    context_layer = tf.keras.layers.Concatenate()(layers) 
    dense1 = tf.keras.layers.Dense(num_dense, name='Dense1',activation='relu')(context_layer)
    dense2 = tf.keras.layers.Dense(1, name='Output', activation='sigmoid')(dense1)
    net = tf.keras.Model(inputs,dense2)
    net.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.MeanSquaredError()) 
    return net

In [ ]:
num_inputs = 20
num_dense = 16
net = build_model(num_inputs, num_dense)

In [ ]:
net_checkpoint_filepath = os.path.join(initial_path, r'chekpoints/net')
net.save_weights(net_checkpoint_filepath)

In [ ]:
df = pd.read_csv(os.path.join(db_path, 'dataset.csv'))
pc_names = df['pc_name'].unique()
df = df.set_index(['pc_name', 'codec_id', 'codec_rate']).sort_index()

In [ ]:
icip_pcs = []
for idx, data in tqdm(df.iterrows()):
    icip_pcs.append(icip_pc(idx[0],idx[1],idx[2],data['geometry_bits'],data['mos'],data['mos_ci'],data['relative_path'],data['radius']))       
for pc in tqdm(icip_pcs):
    pc.load_points()
    pc.connect_with_ref(icip_pcs)
    pc.partition()
    pc.load_tree()
for pc in tqdm(icip_pcs):
    pc.load_dists_ngbs()
    pc.compute_features()
    pc.find_shared_blocks()
    pc.downsample(downsampled_size=2048)

In [ ]:
icip_partitions = {}
for name in pc_names:
    test_dic = {}
    train_dic = {}

    ids_train_set = list(set([pc.id for pc in icip_pcs if (pc.pc_name != name and pc.is_ref == False)]))
    for id in ids_train_set :
        id_blocks = []
        for pc in icip_pcs:
          if pc.id == id :
            for block in pc.shared_blocks:
                id_blocks.append([block])
        train_dic[id] = id_blocks

    ids_test_set = list(set([pc.id for pc in icip_pcs if (pc.pc_name == name and pc.is_ref == False)]))
    for id in ids_test_set :
        id_blocks = []
        for pc in icip_pcs:
          if pc.id == id :
            for block in pc.shared_blocks:
                id_blocks.append([block])
        test_dic[id] = id_blocks

    icip_partitions[name] = {'train' : train_dic, 'test' : test_dic}

In [ ]:
def icip_push_sample (num):
    num = num.numpy()
    pc_id = ids[num]
    blocks = train_list[num]
    inputs = []
    for pc in icip_pcs:
      if pc.id == pc_id :
        for block in blocks:
          x1 = pc.blocks_meta[block]['block']
          zeros1 = np.zeros(block_shape, dtype=np.float32)
          x1 = pts_to_vx(x1, block_shape, zeros1)
          x1 = x1.reshape([64,64,64,1])
          x1 = tf.convert_to_tensor(x1, dtype=tf.float32)
          x2 = pc.ref.blocks_meta[block]['block']
          zeros2 = np.zeros(block_shape, dtype=np.float32)
          x2 = pts_to_vx(x2, block_shape, zeros2)
          x2 = x2.reshape([64,64,64,1])
          x2 = tf.convert_to_tensor(x2, dtype=tf.float32)
          inputs.append(x1)
          inputs.append(x2)
        break
    for pc in icip_pcs:
        if pc.id == pc_id :
            inputs.append(pc.downsampled)
            break
    inputs = tuple(inputs)
    return inputs
def tf_icip_push_sample (num):
    return tf.py_function(icip_push_sample, [num], ((num_inputs*2)+1) * [tf.float32])
def icip_divide_sample (x1, d2):
    return (x1, d2)
def push_mos(num):
    num = num.numpy()
    pc_id = ids[num]
    blocks = train_list[num]
    for pc in icip_pcs:
        if pc.id == pc_id :
            mos = pc.mos/5
            break
    return mos
def tf_push_mos (num):
    return tf.py_function(push_mos, [num], [tf.float32])

def test_icip_push_sample (num):
    num = num.numpy()
    pc_id = ids[num]
    blocks = test_list[num]
    inputs = []
    for pc in icip_pcs:
      if pc.id == pc_id :
        for block in blocks:
          x1 = pc.blocks_meta[block]['block']
          zeros1 = np.zeros(block_shape, dtype=np.float32)
          x1 = pts_to_vx(x1, block_shape, zeros1)
          x1 = x1.reshape([1,64,64,64,1])
          x1 = tf.convert_to_tensor(x1, dtype=tf.float32)
          x2 = pc.ref.blocks_meta[block]['block']
          zeros2 = np.zeros(block_shape, dtype=np.float32)
          x2 = pts_to_vx(x2, block_shape, zeros2)
          x2 = x2.reshape([1,64,64,64,1])
          x2 = tf.convert_to_tensor(x2, dtype=tf.float32)
          inputs.append(x1)
          inputs.append(x2)
        break
    for pc in icip_pcs:
        if pc.id == pc_id :
            inputs.append(pc.downsampled.reshape([1,2048,3]))
            break
    inputs = tuple(inputs)
    return inputs
def tf_test_icip_push_sample (num):
    return tf.py_function(test_icip_push_sample, [num], ((num_inputs*2)+1) * [tf.float32])

In [ ]:
for name in tqdm(icip_partitions.keys()):
    train_list = []
    ids = list(icip_partitions[name]['train'].keys())
    for id in ids :
      chosen_idxs = np.random.choice(range(len(list(icip_partitions[name]['train'][id]))), num_inputs, replace = False)
      chosen_blocks = [list(icip_partitions[name]['train'][id])[x][0] for x in chosen_idxs]
      train_list.append(chosen_blocks)

    pre_dataset = tf.data.Dataset.range(75)

    icip_dataset = pre_dataset.map(tf_icip_push_sample, num_parallel_calls = 75)
    mos_dataset =  pre_dataset.map(tf_push_mos)
    dataset = tf.data.Dataset.zip((icip_dataset, mos_dataset))
    dataset = dataset.shuffle(75)
    train_dataset = dataset.take(70)
    train_dataset =train_dataset.batch(1).prefetch(1)
    val_dataset = dataset.skip(70)
    val_dataset = dataset.batch(1).prefetch(1)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
    net.load_weights(net_checkpoint_filepath)
    net.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss=tf.keras.losses.MeanSquaredError())
    net.fit(train_dataset, epochs=100, callbacks=[callback,reduce_lr], initial_epoch=0 ,validation_data=val_dataset)
    
    test_list = []
    ids = list(icip_partitions[name]['test'].keys())
    for id in ids :
      chosen_idxs = np.random.choice(range(len(list(icip_partitions[name]['test'][id]))), num_inputs, replace = False)
      chosen_blocks = [list(icip_partitions[name]['test'][id])[x][0] for x in chosen_idxs]
      test_list.append(chosen_blocks)

    pre_dataset = tf.data.Dataset.range(len(ids))
    test_dataset = pre_dataset.map(tf_test_icip_push_sample, num_parallel_calls = len(ids))
    predictions = []
    for i, elem in enumerate(list(test_dataset.as_numpy_iterator())) :
      id = ids[i]
      for pc in icip_pcs:
        if pc.id == id:
          pc.predicted_mos = net(elem)
          break

In [ ]:
mos_list = np.reshape(np.asarray([pc.mos for pc in icip_pcs]), -1)
preidctions_list = np.reshape(np.asarray([float(pc.predicted_mos)*5 for pc in icip_pcs]), -1)

In [ ]:
plcc=scipy.stats.pearsonr(mos_list, preidctions_list)
srocc=scipy.stats.spearmanr(mos_list, preidctions_list)

In [ ]:
print(plcc)
print(srocc)